In [512]:
### Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%pylab
from sklearn.pipeline import make_pipeline
from scipy.signal import argrelextrema
import statsmodels.discrete.discrete_model as sm

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [513]:
df_outliers = pd.read_excel('DistrictTypes.xlsx')

In [514]:
df_outliers.head()

,District,State,Type,DistCode
0,Agra,Uttar Pradesh,4. Large City,146
1,Ahmadabad,Gujarat,1. Metro,474
2,Aizawl,Mizoram,3. Capital,283
3,Ajmer,Rajasthan,4. Large City,119
4,Allahabad,Uttar Pradesh,4. Large City,175


In [515]:
snow_id = df_outliers[df_outliers['Type'] == '5. Snow Clad']['DistCode'].values.tolist()

In [516]:
metro = df_outliers[df_outliers['Type'] == '1. Metro']['DistCode'].values.tolist()

In [517]:
### Import Data Files
df_satData = pd.read_csv('distSatellite_withLabels.csv')

In [518]:
df_asset = pd.get_dummies(pd.read_csv('District_HHD_Cluster_Asset_Labels.csv')[['District','District_HHD_Cluster_Asset']])

In [519]:
df_asset.columns = ['dist_code','asset_1','asset_2','asset_3']

In [520]:
df_satData = (df_satData.merge(df_asset,left_on='101',right_on='dist_code',how='outer'))

In [521]:
del df_satData['dist_code']

In [522]:
df_satData = df_satData[~(df_satData['101'].isin(snow_id+metro))]

In [523]:
### List of labels
lis_labelsAll = ['MSL_1','MSL_2','MSL_3','MSW_1','MSW_2','MSW_3','CHH_1','CHH_2','CHH_3','FC_1','FC_2','FC_3','BF_1','BF_2','BF_3','EMP_1','EMP_2','EMP_3']

### List of useful Labels
lis_labels= ['asset_1','asset_2','asset_3']

In [524]:
df_satData.head()

,DISTRICT,101,DN_00,DN_01,DN_02,DN_03,DN_04,DN_05,DN_06,DN_07,...,FC_3,BF_1,BF_2,BF_3,EMP_1,EMP_2,EMP_3,asset_1,asset_2,asset_3
0,Adilabad,532,5237.12,0,0,0.76,1815.67,3332.38,2043.90,1049.05,...,0,1,0,0,0,1,0,0,1,0
1,Agra,146,401.66,0,0,0.00,48.54,451.36,620.77,441.57,...,1,0,0,1,1,0,0,0,1,0
3,Ahmadnagar,522,984.52,0,0,3.12,1756.19,3855.19,2683.53,1989.69,...,0,1,0,0,0,1,0,0,1,0
4,Aizawl,283,3412.59,0,0,0.00,49.31,102.02,42.99,27.32,...,1,0,0,1,0,0,1,0,0,1
5,Ajmer,119,3903.05,0,0,1.61,386.31,1286.76,648.55,490.42,...,1,0,0,1,0,0,1,0,1,0


In [525]:
df_satData.shape

(592, 106)

In [526]:
cols = df_satData.columns.tolist()

In [527]:
cols_new = cols[2:cols.index('MSL_1')]

In [528]:
df_satData

,DISTRICT,101,DN_00,DN_01,DN_02,DN_03,DN_04,DN_05,DN_06,DN_07,...,FC_3,BF_1,BF_2,BF_3,EMP_1,EMP_2,EMP_3,asset_1,asset_2,asset_3
0,Adilabad,532,5237.12,0,0,0.76,1815.67,3332.38,2043.90,1049.05,...,0,1,0,0,0,1,0,0,1,0
1,Agra,146,401.66,0,0,0.00,48.54,451.36,620.77,441.57,...,1,0,0,1,1,0,0,0,1,0
3,Ahmadnagar,522,984.52,0,0,3.12,1756.19,3855.19,2683.53,1989.69,...,0,1,0,0,0,1,0,0,1,0
4,Aizawl,283,3412.59,0,0,0.00,49.31,102.02,42.99,27.32,...,1,0,0,1,0,0,1,0,0,1
5,Ajmer,119,3903.05,0,0,1.61,386.31,1286.76,648.55,490.42,...,1,0,0,1,0,0,1,0,1,0
6,Akola,501,445.44,0,0,0.00,1088.96,1985.74,948.76,358.79,...,0,1,0,0,1,0,0,1,0,0
7,Alappuzha,598,0.00,0,0,0.00,17.23,54.85,65.70,89.68,...,1,0,1,0,0,0,1,0,1,0
8,Aligarh,143,1382.57,0,0,0.00,104.68,849.94,522.45,248.51,...,0,0,0,1,1,0,0,0,1,0
9,Alirajpur,465,3061.12,0,0,0.00,97.97,106.21,19.07,15.27,...,0,1,0,0,0,1,0,1,0,0
10,Allahabad,175,928.97,0,0,0.00,662.81,1121.60,651.43,431.70,...,1,0,0,1,1,0,0,0,1,0


In [529]:
for i,quantile_val in enumerate(quantile_list):
    df_satData.loc[df_satData[cols_new[i]] > quantile_val,cols_new[i]] = quantile_val

In [254]:
pure_modis_features = cols[cols.index('Water'):cols.index('MSL_1')]

In [533]:
pureSatFeatures = cols[2:cols.index('MSL_1')]

## Building features from night light data

In [581]:
var = []
for i in range(64):
    var.append('DN_' + str(i).zfill(2))

df_satData['Area'] = 0
for i in var:
    df_satData['Area']=df_satData['Area'] + df_satData[i]
df_satData['Area'] = df_satData['Area'].replace(0,1)

for i in var:
    df_satData['p'+i] = (df_satData[i])/(df_satData['Area']*1.0)

for i,j in enumerate(var):
    df_satData['q'+j] = df_satData[j]*(i+1)

var3 = []
for i in var:
    var3.append('p'+i)
for i in var:
    var3.append('q'+i)

In [583]:
df_satData[var3]

,pDN_00,pDN_01,pDN_02,pDN_03,pDN_04,pDN_05,pDN_06,pDN_07,pDN_08,pDN_09,...,qDN_54,qDN_55,qDN_56,qDN_57,qDN_58,qDN_59,qDN_60,qDN_61,qDN_62,qDN_63
0,0.365028,0.0,0.0,0.000053,0.092262,0.172027,0.118932,0.073119,0.045825,0.031578,...,88.55,190.400,235.9800,98.02,139.240,142.2,46.3600,141.36,47.8800,0.00
1,0.099864,0.0,0.0,0.000000,0.012068,0.112221,0.154341,0.109787,0.076473,0.062799,...,328.90,421.568,263.9100,485.46,424.210,417.0,418.4600,886.60,1399.3875,5115.04
3,0.088088,0.0,0.0,0.000279,0.118435,0.220828,0.152671,0.097314,0.067644,0.047501,...,172.15,127.680,274.1700,274.92,424.210,233.4,414.1900,403.62,922.9500,0.00
4,0.914474,0.0,0.0,0.000000,0.013214,0.027338,0.011520,0.007321,0.005330,0.002294,...,0.00,0.000,0.0000,0.00,0.000,0.0,0.0000,0.00,0.0000,0.00
5,0.458254,0.0,0.0,0.000189,0.045356,0.151078,0.076146,0.057580,0.033035,0.026475,...,247.50,301.280,408.1200,506.05,374.060,361.8,519.7200,881.02,921.0600,1331.20
6,0.082385,0.0,0.0,0.000000,0.201406,0.367269,0.175476,0.066359,0.024963,0.013651,...,46.20,47.040,42.7500,92.22,93.810,146.4,92.1100,249.24,153.0900,461.44
7,0.000000,0.0,0.0,0.000000,0.012126,0.038603,0.046239,0.063116,0.085025,0.062159,...,0.00,0.000,0.0000,0.00,0.000,0.0,0.0000,0.00,0.0000,0.00
8,0.369304,0.0,0.0,0.000000,0.027961,0.227031,0.139554,0.066380,0.038061,0.019943,...,43.45,244.160,212.0400,165.30,224.200,85.8,236.6800,451.98,641.3400,730.24
9,0.916289,0.0,0.0,0.000000,0.029325,0.031792,0.005708,0.004571,0.001191,0.000446,...,0.00,0.000,0.0000,0.00,0.000,0.0,0.0000,0.00,0.0000,0.00
10,0.178073,0.0,0.0,0.000000,0.127053,0.214998,0.124872,0.082752,0.064409,0.036770,...,168.85,339.920,262.7700,399.04,373.470,233.4,365.3900,438.96,770.4900,2910.72


In [585]:
from sklearn.cluster import KMeans

In [586]:
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_satData[var3])
    #print(data["clusters"])
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [596]:
kmeans = KMeans(n_clusters=3, max_iter=1000).fit(df_satData[var3].T)
df_ntlBins = pd.DataFrame(kmeans.labels_)

## Appling Pca to matrix formed from features of nightlight data

In [564]:
X = df_satData.loc[:,var3].values

In [565]:
X = preprocessing.scale(X)
pca = PCA()
pca.fit(X)

#pca= PCA(svd_solver='full')
#pca.fit(X, y=None )

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

#Number of PC comprising 95%Variance
n = len(var1)-len([num for num in var1 if num != 0 and num > 85.0])+1
print( "n: 95% Variance: ",n)

[ 43.44  61.91  69.68  73.81  77.19  79.52  81.53  83.45  85.03  86.3
  87.43  88.44  89.21  89.94  90.52  91.05  91.54  92.02  92.41  92.76
  93.08  93.38  93.66  93.92  94.16  94.4   94.63  94.85  95.07  95.27
  95.47  95.66  95.84  96.02  96.19  96.36  96.52  96.68  96.83  96.98
  97.12  97.26  97.39  97.52  97.64  97.76  97.87  97.98  98.08  98.18
  98.28  98.38  98.47  98.56  98.64  98.72  98.8   98.87  98.94  99.01
  99.07  99.13  99.19  99.25  99.3   99.35  99.4   99.45  99.49  99.53
  99.57  99.6   99.63  99.66  99.69  99.71  99.73  99.75  99.77  99.79
  99.81  99.82  99.83  99.84  99.85  99.86  99.87  99.88  99.89  99.9
  99.91  99.92  99.93  99.94  99.95  99.96  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97
  99.97  99.97  99.97  99.97  99.97  99.97  99.97  99.97]
('n: 95% Variance: ', 9)


In [566]:
### PCA fit
X=pca.fit_transform(X)
X=X[:,0:n]

In [567]:
n

9

In [568]:
## creating variance table 

In [574]:
df_pcaComp = pd.DataFrame(pca.components_)

In [575]:
pca_idx = ['PCA_'+str(i) for i in range(len(df_pcaComp))]

In [576]:
df_pcaComp.index = pca_idx

In [577]:
df_pcaComp.columns = var3

In [580]:
df_pcaComp[:9].to_csv('pcaVarTableRaw.csv')

In [ ]:
bins_pcaVar = [[0]*10]*7

for column in df_pcaComp:
    bins_pcaVar[(int(column[4:])/10)] += (abs(df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1))

bins_pcaVar[5] += bins_pcaVar[6]

In [ ]:
df_bins_pcaVar = pd.DataFrame(bins_pcaVar[:6]).T

In [ ]:
df_bins_pcaVar.columns = ['L1','L2','M1','M2','H1','H2']

In [ ]:
df_bins_pcaVar

In [ ]:
df_newVarTable = pd.DataFrame(columns=['0','1','2','3','4','5'])

for i in range(10):
    temp = df_bins_pcaVar[i:i+1].sort_values(by=pca_idx[i],axis=1,ascending=False)
    temp_cols = temp.columns.tolist()
    temp_vals = np.round(temp.values[0],2).tolist()
    new_row = [str(temp_vals[j])+','+temp_cols[j] for j in range(len(temp_cols))]
    df_newVarTable.loc[pca_idx[i],:] = new_row
    

In [ ]:
#df_newVarTable.to_csv('newVarTable.csv')

In [266]:
### Creating Dataframe of PCA
col_X = []
for j in range(n):
    col_X.append('PCA_'+str(j))
df_pcaNTL = pd.DataFrame(X,columns=col_X)

In [267]:
df_pcaNTL['101'] = df_satData['101'].values.tolist()

In [268]:
df_pcaNTL.head()

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,101
0,4.234774,-5.881810,-1.887409,2.798423,2.458637,-0.017588,0.975029,2.632625,1.681116,532
1,10.932850,-2.143393,-1.732868,-0.964789,-1.694221,0.733920,-0.026523,-2.311400,-1.420471,146
2,10.882314,-8.194137,-1.974385,3.489951,2.060851,0.621417,1.157466,3.520213,-0.693791,522
3,-5.778448,0.913611,-0.011275,-1.384105,-1.041271,-0.667338,0.640582,0.145919,0.919681,283
4,9.837009,-4.363057,-5.151333,-3.081372,-0.583621,-0.309553,0.795833,0.366711,-0.379171,119


## Building features out of Modis Data

In [260]:
df_satData['Urban and built-up']=df_satData['Urban and built-up']+1
df_satData['CropRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/df_satData['Area']
df_satData['AvgUrbanNTL']=df_satData['sum']/df_satData['Urban and built-up']
df_satData['UrbanRatio']= df_satData['Urban and built-up']/df_satData['Area']

df_satData['Natural']= 0
for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands','Snow and ice']:
    df_satData['Natural']=df_satData['Natural']+df_satData[i]
    
df_satData['CropRemainRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/(df_satData['Area']-df_satData['Natural'])
df_satData['UrbanRemainRatio']=df_satData['Urban and built-up']/(df_satData['Area']-df_satData['Natural'])
#df_satData['UrbanRatio']=df_satData['UrbanRatio'].fillna(0)


forest =['Evergreen Broadleaf forest','Deciduous Broadleaf forest','Mixed forest']
df_satData['forest'] = df_satData[forest].sum(axis=1)

grass_shrubs =['Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands']
df_satData['grass_shrubs'] = df_satData[grass_shrubs].sum(axis=1)


In [261]:
modi_area = df_satData[pure_modis_features].sum(axis=1).values.tolist()

In [262]:
left_area = [df_satData['Area'].values.tolist()[i]-modi_area[i] for i in range(len(df_satData))]

In [263]:
df_satData['left_area'] = left_area

In [264]:
modi_var = []
modi_var=['mean','sum','CropRatio', 'AvgUrbanNTL', 'UrbanRatio', 'CropRemainRatio', 'UrbanRemainRatio','Natural']
for i in ['Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'forest',
 'grass_shrubs']:
    df_satData['mod_'+i]=df_satData[i]/df_satData['Area']
    modi_var.append('mod_'+i)
    modi_var.append(i)
#modi_var.append('left_area')

## final dataframe from pca version of nl and modi 

In [486]:
## only nighlight
df_modi_nlPca =  pd.merge(df_satData[['101','mean','sum']],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [494]:
df_modi_nlPca =  pd.merge(df_satData[['101']+modi_var],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [497]:
## for modis only
df_modi_nlPca = df_satData[['101'] + modi_var]

In [498]:
df_modi_nlPca.shape

(592, 19)

In [499]:
var = df_modi_nlPca.columns.tolist()[5:]

In [500]:
var

['UrbanRatio',
 'CropRemainRatio',
 'UrbanRemainRatio',
 'Natural',
 'mod_Croplands',
 'Croplands',
 'mod_Urban and built-up',
 'Urban and built-up',
 'mod_Cropland/Natural vegetation mosaic',
 'Cropland/Natural vegetation mosaic',
 'mod_forest',
 'forest',
 'mod_grass_shrubs',
 'grass_shrubs']

In [404]:
var.extend(['CropRatio'])

In [506]:
var

['UrbanRatio',
 'CropRemainRatio',
 'UrbanRemainRatio',
 'Natural',
 'mod_Croplands',
 'Croplands',
 'mod_Urban and built-up',
 'Urban and built-up',
 'mod_Cropland/Natural vegetation mosaic',
 'Cropland/Natural vegetation mosaic',
 'mod_forest',
 'forest',
 'mod_grass_shrubs',
 'grass_shrubs']

## finding feature ranking for a particular label

In [502]:
def featureRank(label):
    model = LogisticRegression()
    selector = RFE(model,1, step=1)

    X = df_modi_nlPca[var].values
    X = preprocessing.scale(X)

    y = df_satData.loc[:,label].values

    selector.fit(X,y)

    selectedIdx = selector.get_support(indices=True)

    featureRanking = selector.ranking_

    ranked_var = pd.DataFrame(featureRanking,var)
    ranked_var = ranked_var.reset_index()
    ranked_var.columns = (['var','rank'])
    ranked_var = ranked_var.sort_values('rank')
    ranked_var.reset_index(inplace=True)
    del ranked_var['index']
    
    return ranked_var

## selecting features to maximize f1_score
#### Assumption: Set of first n features in feature ranking will produce maximum f1_score compared to any other set of n features.

In [503]:
def selectFeatures(ranked_var,label):
    metric_list = list()
    f1_list = list()
    for i in range(len(ranked_var)):
        var2 = ranked_var.loc[:i,'var'].values.tolist()

        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values

        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)

        acc = metrics.accuracy_score(y,predicted)
        recall = metrics.recall_score(y,predicted)
        prec =  metrics.precision_score(y,predicted)

        f1 = metrics.f1_score(y,predicted)

        metric_list.append([f1,acc,recall,prec])
        f1_list.append(f1)

    df_metric = pd.DataFrame(metric_list)
    df_metric.columns =  (['f1','acc','recall','prec'])

    ## finding the global maximum index in f1_list
    global_maxima = f1_list.index(max(f1_list))

    ## plotting f1_score with features selected
    #figure()
    #plt.grid()
    #plt.plot(f1_list)
    #plt.title(label)
    #plt.xlabel('number of variables')
    #plt.ylabel('f1_score')
    
    ## findi  ng indexes of all local maximums

    x = array(f1_list)
    # for local maxima
    local_maxima = argrelextrema(x, np.greater)[0].tolist()

    f1_acceptable = f1_list[global_maxima]-0.01

    i = 0
    while f1_list[i] < f1_acceptable:
        i += 1

    var_final = ranked_var.loc[:i,'var']

    X = df_modi_nlPca.loc[:,var_final].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression()

    fit = model.fit(X,y)

    df_selecedVar = ranked_var.loc[:i].copy(deep=True)
    df_selecedVar.insert(2,'coeff',fit.coef_[0])

    return [df_selecedVar,f1_list[i],df_metric[i:i+1]]

In [504]:
df_overallScore = pd.DataFrame(columns=['f1','acc'])
for main_label in ['MSL','MSW','CHH','FC','BF','EMP']:
    lis_dis_label = [main_label+'_'+str(i) for i in range(1,4)]
    yTrue = []
    yPred = []
    for label in lis_dis_label:
        ranked_var = featureRank(label)
        feature_and_score = selectFeatures(ranked_var,label)
        var2 = feature_and_score[0]['var'].values.tolist()
        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values
        yTrue.extend(y)
        
        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)
        yPred.extend(predicted)
    acc = metrics.accuracy_score(yTrue,yPred)
    f1 = metrics.f1_score(yTrue,yPred)  
    
    df_overallScore.loc[main_label] = [f1,acc]

In [505]:
np.round(df_overallScore.sort_values(by='f1',ascending=False),2)#.to_csv('overallScore.csv')

,f1,acc
FC,0.72,0.80
BF,0.70,0.78
MSW,0.67,0.76
EMP,0.66,0.74
CHH,0.62,0.70
MSL,0.58,0.68


In [432]:
label = lis_dis_label[0]

In [417]:
metric_list = list()
f1_list = list()
for i in range(len(ranked_var)):
    var2 = ranked_var.loc[:i,'var'].values.tolist()

    X = df_modi_nlPca.loc[:,var2].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression(class_weight='balanced')
    clf = make_pipeline(preprocessing.StandardScaler(),model)
    predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)


'EMP'

In [424]:
for label in lis_dis_label:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)

In [439]:
var_temp = feature_and_score[0]['var'].values.tolist()

['UrbanRemainRatio',
 'PCA_0',
 'PCA_1',
 'mod_Urban and built-up',
 'mod_Croplands']

In [427]:
select_feature = []
score_metric = []
for label in lis_labelsAll:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)
    select_feature.append(feature_and_score)
    score_metric.append(feature_and_score[2].values[0])

In [429]:
df_scoreMetric = pd.DataFrame(score_metric,columns = ['f1','acc','recall','prec'], index=lis_labelsAll)

In [ ]:
#df_scoreMetric.to_csv('scoreMetric_asset.csv')

In [430]:
np.round(df_scoreMetric.sort_values(by='f1',ascending=False),2)

,f1,acc,recall,prec
MSL_3,0.85,0.84,0.84,0.86
FC_1,0.84,0.83,0.84,0.84
BF_1,0.84,0.82,0.84,0.83
MSW_2,0.81,0.82,0.84,0.77
FC_3,0.80,0.89,0.86,0.75
MSW_3,0.78,0.81,0.83,0.73
MSL_1,0.78,0.88,0.91,0.68
EMP_3,0.75,0.87,0.80,0.70
CHH_1,0.73,0.83,0.78,0.68
BF_3,0.72,0.82,0.78,0.67


In [ ]:
select_feature[0][0]

In [ ]:
## adding pvalues.... to the list of featues.

In [ ]:
X = df_modi_nlPca.copy(deep=True)
X.insert(1,'intercept',1.0)
cols = X.columns.tolist()

In [ ]:
modi_cols = cols[1:cols.index('PCA_0')]
pca_cols = ['intercept'] + cols[cols.index('PCA_0'):]

In [ ]:
def find_summary(cols,label):
    X1 = X[cols].values
    y = df_satData.loc[:,label].values

    model = sm.Logit(y,X1)
    result = model.fit()
    summary = result.summary()

    df_summary = pd.DataFrame(result.params,result.pvalues)
    df_summary.reset_index(inplace=True)
    df_summary.index = cols
    df_summary.columns = ['pvals','param']
    
    return df_summary

In [ ]:
selectedFeatures_pval = []
for i,label in enumerate(lis_labelsAll):
    df_temp = select_feature[i][0].copy(deep=True)
    df_temp.index = df_temp['var']
    del df_temp['var']
    df_summary = pd.concat([find_summary(modi_cols,label),find_summary(pca_cols,label)])
    df_temp2 = pd.concat([df_temp,df_summary],axis=1,join_axes=[df_temp.index])
    selectedFeatures_pval.append(df_temp2)

In [ ]:
label = 'BF_1'
selectedFeatures_pval[lis_labelsAll.index(label)]